In [1]:
#import neccessary libraries
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import emoji
import re

In [2]:
#load in the train data
train = pd.read_csv('jigsaw-toxic-comment-classification-challenge/train.csv')
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
#load in the test data
test = pd.read_csv('jigsaw-toxic-comment-classification-challenge/test.csv')
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [4]:
#load in the test labels
test_labels = pd.read_csv('jigsaw-toxic-comment-classification-challenge/test_labels.csv')
test_labels.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [5]:
len(test_labels)

153164

In [6]:
#remove unneccesary rows
test_labels = test_labels[~test_labels['toxic']<0]

In [7]:
#combine the test data and its labels
test = pd.merge(test, test_labels, on='id')

In [8]:
len(train), len(test)

(159571, 63978)

In [9]:
#combine train and test
df = pd.concat([train,test]).reset_index().drop(columns='index')

In [10]:
df.loc[0]['comment_text']

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [11]:
#search for all features that are 0
df[(df['toxic']==0)&(df['severe_toxic']==0)&(df['obscene']==0)&(df['threat']==0)&(df['insult']==0)&(df['identity_hate']==0)]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
223543,fff8f521a7dbcd47,shut down the mexican border withought looking...,0,0,0,0,0,0
223544,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0
223545,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0
223546,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0


In [12]:
df.loc[59]['comment_text']

"Why can't you believe how fat Artie is? Did you see him on his recent appearence on the Tonight Show with Jay Leno? He looks absolutely AWFUL! If I had to put money on it, I'd say that Artie Lange is a can't miss candidate for the 2007 Dead pool!   \n\n  \nKindly keep your malicious fingers off of my above comment, . Everytime you remove it, I will repost it!!!"

In [13]:
#create a condition to be used to create the labels
conditions = [(df['toxic']==0)&(df['severe_toxic']==0)&(df['obscene']==0)&(df['threat']==0)&(df['insult']==0)&(df['identity_hate']==0),
             (df['toxic']==1)&(df['severe_toxic']==0)&(df['obscene']==0)&(df['threat']==0)&(df['insult']==1)&(df['identity_hate']==0),
             (df['toxic']==1)&(df['severe_toxic']==1)&(df['obscene']==1)&(df['threat']==1)&(df['insult']==1)&(df['identity_hate']==1)]

In [14]:
#names of the labels
values = ['non risky', 'potentially risky', 'too risky']

In [15]:
#create the labels based on tehe above conidtions
df['labels'] = np.select(conditions, values)

In [16]:
#distribution of the labels
df['labels'].value_counts()

non risky            201081
0                     20669
potentially risky      1754
too risky                45
Name: labels, dtype: int64

In [17]:
#some of the data did now belong to any category based on the condition, yet they are suppposed to be too risky
df['labels'] = df['labels'].replace({'0':'too risky'})

In [18]:
#distribution of labels
(df['labels'].value_counts(dropna=False, normalize=True))*100

non risky            89.949407
too risky             9.265977
potentially risky     0.784615
Name: labels, dtype: float64

In [19]:
#convert labels into numerical format as requested in the repo
df['labels'] = df['labels'].map({'non risky':'0','potentially risky':'1','too risky':'2'})

In [20]:
#percentage distribution of labels, very imbalanced
(df['labels'].value_counts(dropna=False, normalize=True))*100

0    89.949407
2     9.265977
1     0.784615
Name: labels, dtype: float64

### Clean sentence Training

#### Checking for missing values

In [21]:
print(f"How many missing values are there? {df.isnull().any().sum()}")

How many missing values are there? 0


#### stopwords and lemmatizing

In [22]:
#instantiate lemmatizer
lemmatizer = WordNetLemmatizer()

In [23]:
def clean_texts(text):
    """Removes digits, words containing digits, punctuations, special characters, extra spaces and links from text

    Args:
        text (str): raw texts
    Returns:
        text (str) : cleaned texts
    """
    #removes links
    text = re.sub(r'http*\S+', '', text)
    #removes digits 
    text = re.sub('\d+','', text)
    #removes punctuations and special characters
    text = re.sub('[^a-zA-Z]+', ' ', text)
    return text

In [24]:
df['cleaned_texts'] = df['comment_text'].apply(clean_texts)   

In [25]:
#select only the cleaned_texts column
texts = df['cleaned_texts']
#convert to dataframe
texts = pd.DataFrame(texts)
#create empty list to hold the final output
corpus = []
#loop through each row of comments
for i in range(0, len(texts)):
    #tokenize the text
    review = texts['cleaned_texts'][i].split()
    #remove stopwords and lemmatize
    text_stopwords_lemmatized = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    #combine the tokenized texts
    text_stopwords_lemmatized = ' '.join(text_stopwords_lemmatized)
    #put final result into the corpus list
    corpus.append(text_stopwords_lemmatized)

In [26]:
#create dataframe with the appropriate name as accoridng to repo guidelines
corpus = pd.DataFrame(corpus, columns=['clean_sentence_training'])

In [27]:
#merge the main dataframe with the cleaned one
df = pd.concat([df, corpus], axis=1)

In [47]:
#function to compare the raw texts to the cleaned
def compare(df, index):
    """Compares the raw text to the cleaned text

    Args:
        df (pandas dataframe): dataframe containing raw texts and cleaned texts
        index (int) : position of comment in the dataframe
    Returns:
        None
    """
    print('RAW TEXT')
    print(df.loc[index]['comment_text'])
    print('\n\n')
    print('CLEANED TEXT FOR TRAINING')
    print(df.loc[index]['clean_sentence_training'])

In [48]:
compare(df, 42)

RAW TEXT
You are gay or antisemmitian? 

Archangel WHite Tiger

Meow! Greetingshhh!

Uh, there are two ways, why you do erased my comment about WW2, that holocaust was brutally slaying of Jews and not gays/Gypsys/Slavs/anyone...

1 - If you are anti-semitian, than shave your head bald and go to the skinhead meetings!

2 - If you doubt words of the Bible, that homosexuality is a deadly sin, make a pentagram tatoo on your forehead go to the satanistic masses with your gay pals!

3 - First and last warning, you fucking gay - I won't appreciate if any more nazi shwain would write in my page! I don't wish to talk to you anymore!

Beware of the Dark Side!



CLEANED TEXT FOR TRAINING
You gay antisemmitian Archangel WHite Tiger Meow Greetingshhh Uh two way erased comment WW holocaust brutally slaying Jews gay Gypsys Slavs anyone If anti semitian shave head bald go skinhead meeting If doubt word Bible homosexuality deadly sin make pentagram tatoo forehead go satanistic mass gay pal First last 

In [49]:
#function to check for emojis
def extract_emojis(s):
    """Checks if there are any emojis present

    Args:
        s (pandas series): column containing the texts
    Returns:
        result (list) : list containing emoji values if present, if not, empty string
    """    
    result= ''.join(c for c in s if c in emoji.EMOJI_DATA)
    return result

In [50]:
extract_emojis(df['comment_text'])

''

In [51]:
#rename columns to match the appropriate format requested in the repo
df = df.rename(columns={'comment_text':'raw_sentence','labels':'label'})

In [52]:
#make a column called corpus that contains the text Toxic Comment Classification Challenge
df = df.assign(corpus_name = lambda x:'Toxic Comment Classification Challenge')

In [53]:
df.head(2)

,id,raw_sentence,toxic,severe_toxic,obscene,threat,insult,identity_hate,label,cleaned_texts,clean_sentence_training,corpus_name
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0,Explanation Why the edits made under my userna...,Explanation Why edits made username Hardcore M...,Toxic Comment Classification Challenge
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0,D aww He matches this background colour I m se...,D aww He match background colour I seemingly s...,Toxic Comment Classification Challenge


In [54]:
#duplicate the clean_sentence_training, to become clean_sentence_EDA since there were no emojis
df['clean_sentence_EDA'] = df['clean_sentence_training'] 

In [55]:
#select only relevant columns
df = df[['corpus_name','raw_sentence','label','clean_sentence_training','clean_sentence_EDA']]

In [56]:
df

,corpus_name,raw_sentence,label,clean_sentence_training,clean_sentence_EDA
0,Toxic Comment Classification Challenge,Explanation\nWhy the edits made under my usern...,0,Explanation Why edits made username Hardcore M...,Explanation Why edits made username Hardcore M...
1,Toxic Comment Classification Challenge,D'aww! He matches this background colour I'm s...,0,D aww He match background colour I seemingly s...,D aww He match background colour I seemingly s...
2,Toxic Comment Classification Challenge,"Hey man, I'm really not trying to edit war. It...",0,Hey man I really trying edit war It guy consta...,Hey man I really trying edit war It guy consta...
3,Toxic Comment Classification Challenge,"""\nMore\nI can't make any real suggestions on ...",0,More I make real suggestion improvement I wond...,More I make real suggestion improvement I wond...
4,Toxic Comment Classification Challenge,"You, sir, are my hero. Any chance you remember...",0,You sir hero Any chance remember page,You sir hero Any chance remember page
...,...,...,...,...,...
223544,Toxic Comment Classification Challenge,":Jerome, I see you never got around to this…! ...",0,Jerome I see never got around I surprised I lo...,Jerome I see never got around I surprised I lo...
223545,Toxic Comment Classification Challenge,==Lucky bastard== \n http://wikimediafoundatio...,0,Lucky bastard Heh famous I kida envy Congrats,Lucky bastard Heh famous I kida envy Congrats
223546,Toxic Comment Classification Challenge,==shame on you all!!!== \n\n You want to speak...,0,shame You want speak gay romanian,shame You want speak gay romanian
223547,Toxic Comment Classification Challenge,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...


In [ ]:
#saves to disk
#df.to_csv('Toxic Comment Classification Challenge.csv')